In [1]:
import pandas as pd

In [2]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_01.csv')

In [3]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [4]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [5]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [6]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [7]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [8]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,CHW,6/1/2021,1,641313,Tim Anderson,1,Y,SS
1,CHW,6/1/2021,1,663611,Nick Madrigal,2,Y,2B
2,CHW,6/1/2021,1,660162,Yoan Moncada,3,Y,3B
3,CHW,6/1/2021,1,547989,Jose Abreu,4,Y,1B
4,CHW,6/1/2021,1,606213,Yermin Mercedes,5,Y,DH
...,...,...,...,...,...,...,...,...
278,SEA,6/1/2021,1,672284,Jarred Kelenic,6,Y,CF
279,SEA,6/1/2021,1,666211,Taylor Trammell,7,Y,LF
280,SEA,6/1/2021,1,608596,Tom Murphy,8,Y,C
281,SEA,6/1/2021,1,622268,Donnie Walton,9,Y,2B


In [9]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [10]:
pitch_df.drop(456, inplace = True)

In [11]:
matchups

{'CLE': 'CHW',
 'CHW': 'CLE',
 'BAL': 'MIN',
 'MIN': 'BAL',
 'NYY': 'TB',
 'TB': 'NYY',
 'TOR': 'MIA',
 'MIA': 'TOR',
 'CIN': 'PHI',
 'PHI': 'CIN',
 'ATL': 'WAS',
 'WAS': 'ATL',
 'MIL': 'DET',
 'DET': 'MIL',
 'CHC': 'SD',
 'SD': 'CHC',
 'HOU': 'BOS',
 'BOS': 'HOU',
 'KC': 'PIT',
 'PIT': 'KC',
 'COL': 'TEX',
 'TEX': 'COL',
 'ARI': 'NYM',
 'NYM': 'ARI',
 'SF': 'LAA',
 'LAA': 'SF',
 'LAD': 'STL',
 'STL': 'LAD',
 'SEA': 'OAK',
 'OAK': 'SEA'}

In [12]:
pitch_df.loc[pitch_df['Name']== 'Chase DeJong', 'throws']

Series([], Name: throws, dtype: object)

In [13]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    if len(lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values) > 0: 
        opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                                & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
        if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
                opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
                opp_pitcher.append(opp_pitch)
                keep_pitch.append(opp_pitch)
        else:
            del_indexes.append(i)
    else:
        del_indexes.append(i)
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-13-a4b56b56add5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-13-a4b56b56add5>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [14]:
del_indexes

[20, 21, 22, 23, 24, 25, 26, 27, 28, 124, 125, 126, 127, 128, 129, 130, 131]

In [15]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-15-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [16]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
0,CHW,6/1/2021,1,641313,Tim Anderson,1,Y,SS,R,Shane Bieber,...,0.287234,0.315254,0.418440,0.733694,0.131206,0.361809,36.488439,0.610176,0.318434,103.717097
1,CHW,6/1/2021,1,663611,Nick Madrigal,2,Y,2B,R,Shane Bieber,...,0.314433,0.361905,0.371134,0.733039,0.056701,0.338889,27.508805,2.246847,0.326012,109.659886
2,CHW,6/1/2021,1,660162,Yoan Moncada,3,Y,3B,R,Shane Bieber,...,0.244755,0.355030,0.402098,0.757127,0.157343,0.331551,46.292842,5.231298,0.334841,114.867363
3,CHW,6/1/2021,1,547989,Jose Abreu,4,Y,1B,R,Shane Bieber,...,0.304762,0.369014,0.558730,0.927744,0.253968,0.352113,65.508614,22.286282,0.390480,153.037986
4,CHW,6/1/2021,1,606213,Yermin Mercedes,5,Y,DH,R,Shane Bieber,...,0.297521,0.348485,0.454545,0.803030,0.157025,0.340659,19.642671,4.243411,0.349819,127.009075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,SEA,6/1/2021,1,641584,Jake Fraley,5,Y,RF,R,Chris Bassitt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,SEA,6/1/2021,1,672284,Jarred Kelenic,6,Y,CF,R,Chris Bassitt,...,0.119048,0.212766,0.309524,0.522290,0.190476,0.096774,2.660944,-2.818945,0.234324,51.417023
279,SEA,6/1/2021,1,666211,Taylor Trammell,7,Y,LF,R,Chris Bassitt,...,0.196721,0.279412,0.442623,0.722035,0.245902,0.275862,8.151408,0.223059,0.313483,104.760856
280,SEA,6/1/2021,1,608596,Tom Murphy,8,Y,C,R,Chris Bassitt,...,0.132075,0.178571,0.264151,0.442722,0.132075,0.156250,1.470206,-5.059023,0.196329,25.813338


In [17]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [18]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [19]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [20]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
0,CHW,6/1/2021,1,641313,Tim Anderson,1,Y,SS,R,Shane Bieber,...,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033
1,CHW,6/1/2021,1,663611,Nick Madrigal,2,Y,2B,R,Shane Bieber,...,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033
2,CHW,6/1/2021,1,660162,Yoan Moncada,3,Y,3B,R,Shane Bieber,...,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000
3,CHW,6/1/2021,1,547989,Jose Abreu,4,Y,1B,R,Shane Bieber,...,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033
4,CHW,6/1/2021,1,606213,Yermin Mercedes,5,Y,DH,R,Shane Bieber,...,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033,13.168033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,SEA,6/1/2021,1,641584,Jake Fraley,5,Y,RF,R,Chris Bassitt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,SEA,6/1/2021,1,672284,Jarred Kelenic,6,Y,CF,R,Chris Bassitt,...,9.233161,9.233161,9.233161,9.233161,9.233161,9.233161,9.233161,9.233161,9.233161,9.233161
279,SEA,6/1/2021,1,666211,Taylor Trammell,7,Y,LF,R,Chris Bassitt,...,9.233161,9.233161,9.233161,9.233161,9.233161,9.233161,9.233161,9.233161,9.233161,9.233161
280,SEA,6/1/2021,1,608596,Tom Murphy,8,Y,C,R,Chris Bassitt,...,8.545872,8.545872,8.545872,8.545872,8.545872,8.545872,8.545872,8.545872,8.545872,8.545872


In [21]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [22]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [23]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
9,CHW,6/1/2021,1,656302,Dylan Cease,SP,Y,NaN,5,4,...,4.941176,1.004484,0.520179,0.484305,0.968421,5.882353,1.200000,3.081571,18.067651,19.754539
19,CLE,6/1/2021,1,669456,Shane Bieber,SP,Y,NaN,4,5,...,6.086066,1.859375,0.312500,1.546875,0.976431,4.795082,1.398810,4.411765,14.036401,11.524234
29,MIN,6/1/2021,1,501381,Michael Pineda,SP,Y,NaN,4,5,...,4.909091,1.223776,0.279720,0.944056,0.984849,4.636364,1.157895,3.231292,17.015508,19.393242
48,TB,6/1/2021,1,607192,Tyler Glasnow,SP,Y,NaN,3,6,...,10.800000,2.150171,0.655290,1.494881,1.223077,7.285714,1.647887,4.678899,24.789417,19.549698
58,NYY,6/1/2021,1,593334,Domingo German,SP,Y,NaN,5,4,...,7.363636,1.034483,0.172414,0.862069,0.800000,3.681818,0.864865,3.673469,17.459408,15.153955
68,MIA,6/1/2021,1,645261,Sandy Alcantara,SP,Y,NaN,3,6,...,2.875740,1.587444,0.322870,1.264574,1.346341,6.177515,1.779310,3.944637,15.661328,17.484923
78,TOR,6/1/2021,1,592662,Robbie Ray,SP,Y,NaN,1,8,...,19.555556,2.129032,1.118280,1.010753,2.107936,13.333333,2.477157,6.629002,49.985197,40.415755
87,PHI,6/1/2021,1,605400,Aaron Nola,SP,Y,NaN,3,5,...,5.083682,1.645161,0.225806,1.419355,1.078767,4.832636,1.483516,4.154303,13.562520,13.627032
96,CIN,6/1/2021,1,543243,Sonny Gray,SP,Y,NaN,2,6,...,8.400000,1.890000,0.570000,1.320000,1.441341,8.266667,1.981651,4.870849,22.723920,18.049090
105,WAS,6/1/2021,1,544931,Stephen Strasburg,SP,Y,NaN,2,6,...,9.529412,1.164179,0.805970,0.358209,1.034483,6.705882,1.000000,4.864865,33.000130,31.198708


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [24]:
scores = pd.read_csv('../datasets/6_01_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210601,123v,592662,"Ray, Robbie",1.0,0,1.0,28.70,8500.0,TOR,v mia,NaN,5,1
1,20210601,124o,571760,"Heaney, Andrew",1.0,8,1.0,26.65,7700.0,LAA,@ sfo,NaN,8,1
2,20210601,162f,677651,"Garcia, Luis",1.0,0,1.0,26.15,8300.0,HOU,v bos,NaN,5,1
3,20210601,147u,669456,"Bieber, Shane",1.0,0,NaN,25.55,NaN,CLE,v chw,NaN,6,5
4,20210601,133b,608566,"Marquez, German",1.0,9,1.0,25.35,6700.0,COL,v tex,NaN,3,2(F11)


In [25]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [26]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [27]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [28]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [29]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [30]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [31]:
bat_final.to_csv('../merged_data/hitters_6_01.csv')
pitch_final.to_csv('../merged_data/pitchers_6_01.csv')

In [32]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,CHW,6/1/2021,1,641313,Tim Anderson,1,Y,SS,R,Shane Bieber,...,1.0,1,NaN,6.0,NaN,CHW,@ cle,NaN,5,6
1,CHW,6/1/2021,1,663611,Nick Madrigal,2,Y,2B,R,Shane Bieber,...,1.0,2,NaN,5.0,NaN,CHW,@ cle,NaN,5,6
2,CHW,6/1/2021,1,660162,Yoan Moncada,3,Y,3B,R,Shane Bieber,...,1.0,3,NaN,10.0,NaN,CHW,@ cle,NaN,5,6
3,CHW,6/1/2021,1,547989,Jose Abreu,4,Y,1B,R,Shane Bieber,...,1.0,4,NaN,0.0,NaN,CHW,@ cle,NaN,5,6
4,CHW,6/1/2021,1,606213,Yermin Mercedes,5,Y,DH,R,Shane Bieber,...,1.0,5,NaN,0.0,NaN,CHW,@ cle,NaN,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,SEA,6/1/2021,1,571745,Mitch Haniger,2,Y,DH,R,Chris Bassitt,...,1.0,2,7.0,14.0,4900.0,SEA,v oak,NaN,6,12
220,SEA,6/1/2021,1,572122,Kyle Seager,3,Y,3B,R,Chris Bassitt,...,1.0,3,5.0,12.0,4100.0,SEA,v oak,NaN,6,12
221,SEA,6/1/2021,1,664034,Ty France,4,Y,1B,R,Chris Bassitt,...,1.0,4,34.0,13.0,3100.0,SEA,v oak,NaN,6,12
223,SEA,6/1/2021,1,672284,Jarred Kelenic,6,Y,CF,R,Chris Bassitt,...,1.0,6,7.0,0.0,3700.0,SEA,v oak,NaN,6,12


In [33]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,MIN,6/1/2021,1,501381,Michael Pineda,SP,Y,NaN,4,5,...,1.0,0,1.0,-5.45,9200.0,MIN,@ bal,NaN,4,7
1,TB,6/1/2021,1,607192,Tyler Glasnow,SP,Y,NaN,3,6,...,1.0,0,1.0,22.15,9800.0,TAM,@ nyy,NaN,3,5(F11)
2,NYY,6/1/2021,1,593334,Domingo German,SP,Y,NaN,5,4,...,1.0,0,1.0,13.45,9400.0,NYY,v tam,NaN,5,3(F11)
3,MIA,6/1/2021,1,645261,Sandy Alcantara,SP,Y,NaN,3,6,...,1.0,0,1.0,8.10,10000.0,MIA,@ tor,NaN,1,5
4,TOR,6/1/2021,1,592662,Robbie Ray,SP,Y,NaN,1,8,...,1.0,0,1.0,28.70,8500.0,TOR,v mia,NaN,5,1
5,PHI,6/1/2021,1,605400,Aaron Nola,SP,Y,NaN,3,5,...,1.0,9,1.0,7.25,10200.0,PHI,@ cin,NaN,17,3
6,CIN,6/1/2021,1,543243,Sonny Gray,SP,Y,NaN,2,6,...,1.0,9,1.0,16.30,8900.0,CIN,v phi,NaN,3,17
7,WAS,6/1/2021,1,544931,Stephen Strasburg,SP,Y,NaN,2,6,...,1.0,8,1.0,1.20,8600.0,WAS,@ atl,NaN,11,6
8,ATL,6/1/2021,1,608331,Max Fried,SP,Y,NaN,2,6,...,1.0,9,1.0,6.85,7500.0,ATL,v was,NaN,6,11
9,MIL,6/1/2021,1,641778,Eric Lauer,SP,Y,NaN,0,8,...,1.0,9,1.0,-4.90,8400.0,MIL,v det,NaN,7,10
